In [1]:
import numpy as np
from numpy import pi

from qiskit import QuantumCircuit
from qiskit import QuantumRegister
from qiskit import ClassicalRegister

from random_circuit import random_circuit
from qiskit import Aer
from qiskit import execute
from qiskit.providers.aer.noise import NoiseModel
from qiskit.circuit.exceptions import CircuitError
import scipy.linalg as la

from qutip import Qobj

def matprint(mat, fmt="g"):
    """Pretty print a numpy matrix."""

    col_maxes = [
        max([len(("{:" + fmt + "}").format(x)) for x in col]) for col in mat.T
    ]
    for x in mat:
        for i, y in enumerate(x):
            print(("{:" + str(col_maxes[i]) + fmt + "}").format(y), end="  ")
        print("")


def retrieve_unitary_matrix(qcircuit):
    """Retrieve the matrix block-encoded by qcircuit.
    
    Args: 
        qcircuit: qiskit circuit, n_tot_qubit 

    Returns:
        UA: n_tot_qubit matrix.
    """
    backend = Aer.get_backend('unitary_simulator')
    n_tot_qubit = qcircuit.n_qubits
    job = execute(qcircuit, backend)
    result = job.result()
    UA = result.get_unitary(qcircuit)
    # qiskit's column-major order
    UA_Qobj = Qobj(UA, dims=[[2]*n_tot_qubit, [2]*n_tot_qubit])
    # python's row-major order
    UA_Qobj_rowmajor = UA_Qobj.permute(np.arange(0,n_tot_qubit)[::-1])
    UA = np.array(UA_Qobj_rowmajor)
    
    return UA


def retrieve_state(qcircuit):
    """Retrieve the state by qcircuit."""
    backend = Aer.get_backend('statevector_simulator')
    n_tot_qubit = qcircuit.n_qubits
    job = execute(qcircuit, backend)
    result = job.result()
    state = result.get_statevector(qcircuit)
    # qiskit's column-major order
    state_Qobj = Qobj(state, dims=[[2]*n_tot_qubit, [1]])
    # python's row-major order
    state_Qobj_rowmajor = state_Qobj.permute(np.arange(0,n_tot_qubit)[::-1])
    state_rowmajor = np.array(state_Qobj_rowmajor)[:,0]
    
    return state_rowmajor


def scale_noise_model(noise_model, sigma):
    """Retrieve the noise model from qiskit backend and scale it.

    After retrieving the noise model from a backend, create a new
    noise model by scaling the probability.

    We assume that there are only two types of noises:
        
        'roerror': readout error.
            probabilities is of type np.array(np.array()). Each inner
            array is a discrete probability distribution. We assume the
            one closest to 1.0 to be the 'correct' option. Example of
            scaling of the noise:

                prob = [0.90, 0.06, 0.04]

            then prob[0] is identified to be the correct one. then after
            scaling

                prob = [1-(1-0.90)*sigma, 0.06*sigma, 0.04*sigma]

        'qerror': standard quantum error. There are two possibilities:
            'kraus': a Kraus operator. It is already a quantum channel,
                and therefore 'probabilities' is always [1.0]. This type
                of error mode is discarded at the moment.

            not 'kraus': scale it the same way as the 'roerror'.
            
    Args:
        noise_model: noise_model obtained from a quantum backend, or a
            custom noise_model
        sigma : scaling factor for the noise. 0 <= sigma <= 1
            When sigma = 1.0, the result should be the same as the noise
                model retrieved from the backend without 'kraus'.
            When sigma = 0.0, the result should be noiseless

    Returns:
        noise_model_scaled: scaled noise model. 
    """
    # dump out the noise model
    noise_dict = noise_model.to_dict()

    new_noise_dict = {
            'errors': [],
            'x90_gates': [],
            }

    for ierr in range(0,len(noise_dict['errors'])):
        err = noise_dict['errors'][ierr]
        if err['type'] == 'roerror':
            assert err['operations'] == ['measure']
            probs = err['probabilities']
            for iprob in range(0, len(probs)):
                # Note that this directly modifies the value of probs
                prob = probs[iprob]
                imax = np.argmax(prob)
                for idx in range(0,len(prob)):
                    if idx != imax:
                        prob[idx] *= sigma
                prob[imax] = 1.0-(1.0-prob[imax])*sigma
            new_noise_dict['errors'].append(err)

        elif err['type'] == 'qerror': 
            prob = err['probabilities']
            if prob == [1.0]:
                # This is a Kraus opeartor.
                # https://qiskit.org/documentation/stubs/qiskit.quantum_info.Kraus.html
                assert True

            else:
                # other errors, the same treatment as roerror
                assert len(prob) > 1
                imax = np.argmax(prob)
                for idx in range(0,len(prob)):
                    if idx != imax:
                        prob[idx] *= sigma
                prob[imax] = 1.0-(1.0-prob[imax])*sigma
                new_noise_dict['errors'].append(err)

    new_noise_model = NoiseModel.from_dict(new_noise_dict)

    return new_noise_model

class BlockEncoding(object):
    """Implementation of a RACBEM via Qiskit."""

    def __init__(self, n_be_qubit, n_sys_qubit):
        self.n_be_qubit = n_be_qubit
        self.n_sys_qubit = n_sys_qubit
        self.n_tot_qubit = n_be_qubit + n_sys_qubit
        self.qregs = QuantumRegister(self.n_tot_qubit, 'q')


    def build_random_circuit(self, n_depth, coupling_map=None, basis_gates=None, prob_one_q_op=0.5):
        """Build a random circuit as the block-encoding U_A.
        Args:
            n_depth (int):                      the number of the layers of operations
            coupling_map(CouplingMap or list):  coupling map specifies allowed CNOTs
                default:                        fully connected graph coupling n_qubits
            basis_gates (list):                 the list of labels of basis gates used in construction
                default:                        available gates in gate_set
            prob_one_q_op (float):              the probability of selecting a one-qubit operation when two_q_op is allowed
                default:                        equal probability 0.5
        """
        if hasattr(self, 'qc'):
            self.qc.data.clear()
        self.qc = random_circuit(self.n_tot_qubit, n_depth,
                coupling_map=coupling_map, basis_gates=basis_gates, prob_one_q_op=prob_one_q_op)
        self.qc.name = 'UA  '

    def build_dag(self):
        """Build the circuit for U_A^{\dagger}."""
        assert hasattr(self, 'qc')
        if hasattr(self, 'qc_dag'):
            self.qc_dag.data.clear()
        self.qc_dag = self.qc.inverse()

    def build_measure_circuit(self):
        """Build a circuit that can be used to measure the success
        probability of block encoding"""
        qr = self.qregs
        cr = ClassicalRegister(self.n_be_qubit, 'c')
        qcircuit = QuantumCircuit(qr, cr)
        qcircuit.append(self.qc.to_instruction(), qr)

        # measurements
        qcircuit.measure(qr[0], cr[0])

        return qcircuit

class QSPCircuit(object):
    """Build a QSP circuit.

    Attributes:
        qcircuit: QSP circuit.
        qregs: quantum registers
        cregs: classical registers
    """
    def __init__(self, n_sig_qubit, n_be_qubit, n_sys_qubit):
        self.n_sig_qubit = n_sig_qubit
        self.n_be_qubit = n_be_qubit
        self.n_sys_qubit = n_sys_qubit
        self.n_tot_qubit = n_sig_qubit + n_be_qubit + n_sys_qubit
        self.qregs = QuantumRegister(self.n_tot_qubit, 'q')
        self.cregs = ClassicalRegister(self.n_sig_qubit + self.n_be_qubit, 'c')

        self.qcircuit = QuantumCircuit(self.qregs, self.cregs, name='QSP')

        # so far the code only works with one OR two block encoding qubit due
        # to the implementation of the multi-qubit Toffoli gate.
        assert n_be_qubit == 1 or n_be_qubit == 2
        # so far the code only works if the number of signal qubit is 1
        # (i.e. no LCU yet)
        assert n_sig_qubit == 1

    def build_control_rotation(self, phi):
        """Build the controlled rotation gate."""

        qr = QuantumRegister(self.n_sig_qubit + self.n_be_qubit)
        qc_crot = QuantumCircuit(qr, name='CR\n%.1f'%(phi))

        # Add the X gate to perform control-0.
        if self.n_be_qubit == 1:
            qc_crot.x(qr[1])
            qc_crot.cx(qr[1], qr[0])
            qc_crot.rz(phi * 2., qr[0])
            qc_crot.cx(qr[1], qr[0])
            qc_crot.x(qr[1])
        elif self.n_be_qubit == 2:
            qc_crot.x(qr[1])
            qc_crot.x(qr[2])
            qc_crot.ccx(qr[1], qr[2], qr[0])
            qc_crot.rz(phi * 2., qr[0])
            qc_crot.ccx(qr[1], qr[2], qr[0])
            qc_crot.x(qr[1])
            qc_crot.x(qr[2])          

        return qc_crot
    
    def build_circuit(self, be_qc, be_qc_dag, phi_seq, realpart=True, measure=False, init_prepare=None):
        """Build a QSP circuit.

        Build a circuit for quantum signal processing, given a
        block-encoding matrix. 

        Args:
            be_qc: quantum circuit for block encoding. Can be a compiled
                version
            be_qc_dag: quantum circuit for the inverse of block
                encoding. Can be a compiled version. 
            phi_seq: a sequence of phase factors defined in the QSP
                circuit.
            realpart: if True, returns the real part of the polynomial
                encoded by QSP. This is implemented without the need of
                an extra ancilla qubit.
            measure: if True, adds measurements on the sig_ and be_qubit
            init_prepare: if not None (default), qcircuit will start from
                a prepare circuit on system qubits. Multiple formats are 
                supported
                #. QuantumCircuit
                #. str: string of 0 and 1 specifying state wrt Z basis

        Returns:
            None. This function provides a circuit in self.qcircuit .
        """
        if hasattr(self, 'qcircuit'):
            self.qcircuit.data.clear()
        qr = self.qregs

        if init_prepare is not None:
            if isinstance(init_prepare, QuantumCircuit):
                assert init_prepare.n_qubits + 2 == self.n_tot_qubit
                self.qcircuit.append(init_prepare.to_instruction(), qr[2:])
            elif isinstance(init_prepare, str):
                assert len(init_prepare) + 2 == self.n_tot_qubit
                for bitpos in range(len(init_prepare)):
                    if init_prepare[bitpos] == '1':
                        self.qcircuit.x(qr[bitpos+2])
            else:
                raise CircuitError("only support QuantumCircuit or str object as init_prepare")
            self.qcircuit.barrier()

        dag = False
        if realpart:
            # Add Hadamard gate as prepare oracle
            self.qcircuit.h(qr[0])
        self.qcircuit.barrier()

        # The for loop starts from the last phase factor, starting from
        # UA instead of UA_dag
        qc_crot = self.build_control_rotation(phi_seq[-1])
        self.qcircuit.append(qc_crot.to_instruction(), qr[0:self.n_sig_qubit+self.n_be_qubit])
        self.qcircuit.barrier()
        for phi in reversed(phi_seq[:-1]):
            if not dag:
                self.qcircuit.append(be_qc.to_instruction(), qr[1:])
            else:
                self.qcircuit.append(be_qc_dag.to_instruction(), qr[1:])
            self.qcircuit.barrier()

            if realpart:
                # Add a Z gate before the control. 
                self.qcircuit.z(qr[0])

            qc_crot = self.build_control_rotation(phi)
            self.qcircuit.append(qc_crot.to_instruction(), qr[0:self.n_sig_qubit+self.n_be_qubit])
            self.qcircuit.barrier()

            dag = not dag

        if realpart:
            # Add Hadamard gate as prepare oracle
            self.qcircuit.h(qr[0])
        
        # neglecting the global phase for now

        # measurements. For statevector_simulation and
        # unitary_simulation, measure should be set to False
        if measure:
            cr = self.cregs
            self.qcircuit.measure(qr[0:2], cr[0:2])


class Hermitian_BlockEncoding(object):
    """Implementation of a Hermitian-RACBEM via Qiskit.

    Attributes:
        n_be_qubit, n_sys_qubit -- partition of qubits
        cnd_num -- condition number
        ab -- a,b in h(x) = a*x^2+b
        shift -- constant when shifting spectrum A + c I
        phi_seq -- phi sequence used in quadratic QSVT
    """
    def __init__(self, n_be_qubit, n_sys_qubit):
        # The first be-qubit is excluded as sig_qubit in 
        # quadratic QSVT. Thus, n_be_qubit must >= 2
        assert n_be_qubit >= 2
        self.n_be_qubit = n_be_qubit
        self.n_sys_qubit = n_sys_qubit
        self.n_tot_qubit = n_be_qubit + n_sys_qubit
        self.qregs = QuantumRegister(self.n_tot_qubit, 'q')
        # QSPCircuit object is used to build a generic H-RACBEM
        self.qsp = QSPCircuit(1, n_be_qubit-1, n_sys_qubit)

    def set_ab(self, a, b):
        """Construct phi_seq according to h(x) = a*x^2+b"""
        self.ab = (a, b)
        phi0 = 0.25 * (np.arccos(a+b) + np.arccos(b))
        phi1 = 0.5 * (np.arccos(a+b) - np.arccos(b))
        self.phi_seq = np.array([phi0,phi1,phi0])

    def shift_spectrum(self, c):
        """Shift the spectrum of the Hermitian Block-Encoding
        Let tilde A = h(A), where h(x) = ax^2+b and A is the block-encoding
        this function further shifts tilde A by a constant c
        set (a,b) to output a (1+|c|,2,0)-block-encoding of the
        Hermitian matrix tilde A + c I
        """
        assert hasattr(self, 'ab')
        a = self.ab[0] / (1.0+np.abs(c))
        b = (self.ab[1] + c) / (1.0+np.abs(c))
        self.set_ab(a, b)
        self.shift = c

    def set_cndnum(self, cndnum):
        """Upper bound the condition number of the Hermitian Block-Encoding"""
        self.cndnum = cndnum
        self.set_ab(1-1.0/cndnum, 1.0/cndnum)

    def build_random_circuit(self, n_depth, coupling_map=None, basis_gates=None, prob_one_q_op=0.5):
        """Build a random circuit as the Hermitian block-encoding as U_H.
        
        Args: refer to those in Block-Encoding::build_random_circuit()
        """
        assert hasattr(self, 'phi_seq')
        if hasattr(self, 'qc'):
            self.qc.data.clear()
        # n_tot_qubit-1: exclude the first be_qubit as sig_qubit in quadratic QSVT
        qc = random_circuit(self.n_tot_qubit-1, n_depth,
                coupling_map=coupling_map, basis_gates=basis_gates, prob_one_q_op=prob_one_q_op)
        qc.name = 'UA_0  '
        self.qsp.build_circuit(qc, qc.inverse(), self.phi_seq,
                realpart=True, measure=False)
        self.qc = self.qsp.qcircuit
        self.qc.name = 'UH  '

    def build_dag(self):
        """Build the circuit for U_H^{\dagger}."""
        assert hasattr(self, 'qc')
        if hasattr(self, 'qc_dag'):
            self.qc_dag.data.clear()
        self.qc_dag = self.qc.inverse()

    def build_canonical_hracbem(self, n_depth, coupling_map=None, basis_gates=None, prob_one_q_op=0.5):
        """Build a random circuit as the canonical Hermitian block-encoding as U_H.
        The circuit is shown in Fig.12 in the reference.
        
        Args: refer to those in Block-Encoding::build_random_circuit()
        """
        # so far the code only works with two block encoding qubits due
        # to the implementation of the multi-qubit Toffoli gate.
        assert self.n_be_qubit == 2
        if hasattr(self, 'qc'):
            self.qc.data.clear()
        else:
            self.qc = QuantumCircuit(self.qregs, name='UH  ')
        # n_tot_qubit-1: exclude the first be_qubit as sig_qubit in quadratic QSVT
        qc = random_circuit(self.n_tot_qubit-1, n_depth,
                coupling_map=coupling_map, basis_gates=basis_gates, prob_one_q_op=prob_one_q_op)
        qc.name = 'UA_0  '
        qr = self.qregs
        self.qc.h(qr[0])
        self.qc.t(qr[0])
        self.qc.append(qc.to_instruction(), qr[1:])
        # Perform control-1, do not need to add X gate.
        self.qc.cx(qr[1], qr[0])
        self.qc.sdg(qr[0])
        self.qc.cx(qr[1], qr[0])
        self.qc.append(qc.inverse().to_instruction(), qr[1:])
        self.qc.t(qr[0])
        self.qc.h(qr[0])
        self.qc.name = 'UH  '

In [13]:
Hermitian_BlockEncoding(3,1).build_random_circuit(10)

AssertionError: 

In [25]:
    n_be_qubit=1
    n_sys_qubit=3
    kappa = 5
    n_depth = 8
    n_be_qubit = n_be_qubit + 1     # add one extra qubit as sig_qubit in quadratic QSVT
    be = Hermitian_BlockEncoding(n_be_qubit, n_sys_qubit)
    be.set_cndnum(kappa)
    be.build_random_circuit(n_depth, basis_gates=basis_gates, 
            prob_one_q_op=prob_one_q_op, coupling_map=be_map)
    be.build_dag()
    UA = retrieve_unitary_matrix(be.qc)
    A = UA[0:2**n_sys_qubit, 0:2**n_sys_qubit]
    UA = retrieve_unitary_matrix(be.qc_dag)
    A_dag = UA[0:2**n_sys_qubit, 0:2**n_sys_qubit]
    (svd_U, svd_S, svd_VH) = la.svd(A)
    print("Hermitian RACBEM")
    print("singular value (A) = \n", np.around(svd_S, decimals=3))
    print("condition number (A)  = %.3f"%(svd_S.max()/svd_S.min()))
    print("||A - A^\dagger||_2   = %.3e"%(la.norm(A - A_dag)))

NameError: name 'basis_gates' is not defined

In [2]:
"""Solving QLSP and testing Hermitian RACBEM.

This is a demo code of RACBEM.

This includes a utility function for generating the matrix A (to be
included in the racbem.py) and the utility function for generating
random circuit (to be included in the random_circuit.py).

This tests the success probability of solving QLSP and tests that the 
condition number of the Hermitian RACBEM is upper bounded by the given value.

References:
    Yulong Dong, Lin Lin. Random circuit block-encoded matrix and a proposal 
        of quantum LINPACK benchmark.
Authors:
    Yulong Dong     dongyl (at) berkeley (dot) edu
    Lin Lin         linlin (at) math (dot) berkeley (dot) edu
Version: 1.0
Last revision: 06/2020
"""

import numpy as np
import scipy.linalg as la

from qiskit import execute
from qiskit import Aer
from qiskit import IBMQ
from qiskit.tools.monitor import job_monitor
from qiskit.providers.aer.noise import NoiseModel

from racbem import *

import os
import pickle

def GetBackend(backend_name=None):
    if backend_name == None:
        backend = Aer.get_backend('qasm_simulator')
    else:
        provider = IBMQ.load_account()
        backend = provider.get_backend(backend_name)
    return backend

if __name__ == '__main__':
    backend_name = 'ibmq_burlington'
    #backend_name = None
    kappa = 5                   # condition number
    n_sys_qubit = 3             # the number of system qubits
    n_be_qubit = 1              # the number of block-encoding qubit
    n_sig_qubit = 1             # the number of signal qubit
    n_tot_qubit = n_sig_qubit+n_be_qubit+n_sys_qubit
    n_depth = 15                # the depth of random circuit
    prob_one_q_op = 0.5         # the probability of selecting a one-qubit
                                # operation when two_q_op is allowed
    basis_gates = ['u1','u2','cx']
    digit_shots = 13
    n_shots = 2**digit_shots    # the number of shots used in measurements
    sigma = 1.0                 # parameter used to rescale noise model
    # state |0^n>
    b = np.zeros((2**n_sys_qubit,))
    b[0] = 1.0
    load_architecture = True    # True:     load architure locally
                                # False:    need to save an IBM account beforehand

    # instances of RACBEM
    be = BlockEncoding(n_be_qubit, n_sys_qubit)
    qsp = QSPCircuit(n_sig_qubit, n_be_qubit, n_sys_qubit)

    # retrieve backends and architectures
    backend = GetBackend()
    if load_architecture:
        if os.path.exists(backend_name+'_backend_config.pkl'):
            noise_backend = pickle.load(open(backend_name+'_backend_config.pkl','rb'))
            noise_model = NoiseModel.from_dict(noise_backend['noise_dict'])
            coupling_map = noise_backend['coupling_map']
            tot_q_device = noise_backend['tot_q_device']
            print("load architecture locally at: %s_backend_config.pkl\n"%(backend_name))
        else:
            raise Exception("no locally saved architecture: %s_backend_config.pkl"%(backend_name), load_architecture)
    else:
        noise_backend = GetBackend(backend_name=backend_name)
        coupling_map = noise_backend.configuration().coupling_map
        noise_model = NoiseModel.from_backend(noise_backend)
        tot_q_device = noise_backend.configuration().n_qubits
        pickle.dump({'noise_dict': noise_model.to_dict(), 'coupling_map': coupling_map, 'tot_q_device': tot_q_device, 
                    'basis_gates': noise_backend.configuration().basis_gates}, open(backend_name+'_backend_config.pkl','wb'))
        print("retrieve architecture from IBM Q and save locally at: %s_backend_config.pkl\n"%(backend_name))
    assert tot_q_device >= n_tot_qubit
    new_noise_model = scale_noise_model(noise_model, sigma)

    # exclude qubit 0 as signal qubit, shift the remaining labels by -1
    be_map = [[q[0]-1,q[1]-1] for q in coupling_map if (0 not in q) and 
            (q[0] < n_tot_qubit) and (q[1] < n_tot_qubit)]
    be.build_random_circuit(n_depth, basis_gates=basis_gates, 
            prob_one_q_op=prob_one_q_op, coupling_map=be_map)
    be.build_dag()

    # load phase factors
    data = np.loadtxt("phi_inv_%d.txt"%(kappa))
    phi_seq = data[:-2]
    scale_fac = data[-2]
    app_err = data[-1]

    # retrieve block-encoded matrix
    UA = retrieve_unitary_matrix(be.qc)
    A = UA[0:2**n_sys_qubit, 0:2**n_sys_qubit]
    (svd_U, svd_S, svd_VH) = la.svd(A)
    print("kappa=%d, sigma=%.2f, polynomial approximation error=%.3e"%(kappa, sigma, app_err))
    print("")
    print("Generic RACBEM")
    print("singular value (A) = \n", np.around(svd_S, decimals=3))

    # succ prob via measurement
    qsp.build_circuit(be.qc, be.qc_dag, phi_seq, realpart=True, measure=True)
    compiled_circ = qsp.qcircuit
    job = execute(compiled_circ, backend=backend,
            noise_model=new_noise_model, shots=n_shots)
    job_monitor(job)
    result = job.result()
    counts = result.get_counts(compiled_circ)
    # both the signal and the ancilla qubit for block-encoding needs to
    # be 0
    prob_meas = np.float(counts['00']) / n_shots
    # succ prob via noiseless simulator
    qsp.build_circuit(be.qc, be.qc_dag, phi_seq, realpart=True, measure=False)
    state = retrieve_state(qsp.qcircuit)
    x = state[0:2**n_sys_qubit]
    prob_qsp = la.norm(x)**2
    # exact succ prob
    svd_S_herm = (1-1.0/kappa)*svd_S**2+1.0/kappa
    A_herm_inv = svd_VH.transpose().conjugate() @ np.diag(1/svd_S_herm) @ svd_VH
    x_exact = A_herm_inv @ b / scale_fac
    prob_exact = la.norm(x_exact)**2
    print("succ prob (exact)     = ", prob_exact)
    print("succ prob (noiseless) = ", prob_qsp)
    print("succ prob (measure)   = ", prob_meas)
    print("")

    # instance of Hermitian Block-Encoding
    n_be_qubit = n_be_qubit + 1     # add one extra qubit as sig_qubit in quadratic QSVT
    be = Hermitian_BlockEncoding(n_be_qubit, n_sys_qubit)
    be.set_cndnum(kappa)
    be.build_random_circuit(n_depth, basis_gates=basis_gates, 
            prob_one_q_op=prob_one_q_op, coupling_map=be_map)
    be.build_dag()
    UA = retrieve_unitary_matrix(be.qc)
    A = UA[0:2**n_sys_qubit, 0:2**n_sys_qubit]
    UA = retrieve_unitary_matrix(be.qc_dag)
    A_dag = UA[0:2**n_sys_qubit, 0:2**n_sys_qubit]
    (svd_U, svd_S, svd_VH) = la.svd(A)
    print("Hermitian RACBEM")
    print("singular value (A) = \n", np.around(svd_S, decimals=3))
    print("condition number (A)  = %.3f"%(svd_S.max()/svd_S.min()))
    print("||A - A^\dagger||_2   = %.3e"%(la.norm(A - A_dag)))


load architecture locally at: ibmq_burlington_backend_config.pkl

kappa=5, sigma=1.00, polynomial approximation error=1.902e-02

Generic RACBEM
singular value (A) = 
 [1. 1. 1. 1. 0. 0. 0. 0.]
Job Status: job has successfully run
succ prob (exact)     =  0.4328261648721375
succ prob (noiseless) =  0.41171851552736743
succ prob (measure)   =  0.4266357421875

Hermitian RACBEM
singular value (A) = 
 [0.975 0.975 0.653 0.653 0.547 0.547 0.225 0.225]
condition number (A)  = 4.336
||A - A^\dagger||_2   = 4.935e-15


In [3]:
    print("kappa=%d, sigma=%.2f, polynomial approximation error=%.3e"%(kappa, sigma, app_err))
    print("")
    print("Generic RACBEM")
    print("singular value (A) = \n", np.around(svd_S, decimals=3))

kappa=5, sigma=1.00, polynomial approximation error=1.902e-02

Generic RACBEM
singular value (A) = 
 [0.975 0.975 0.653 0.653 0.547 0.547 0.225 0.225]


In [4]:
from qiskit.visualization import array_to_latex
array_to_latex(UA,precision=10, source=False, max_size=80)

<IPython.core.display.Latex object>

In [7]:
help(array_to_latex)

Help on package array_to_latex:

NAME
    array_to_latex - Return numpy arrays and Pandas dataframes as LaTeX.

DESCRIPTION
    Provides `to_ltx` and `to_clp` which convert numpy arrays and Pandas dataframes
    arrays to LaTeX form.

PACKAGE CONTENTS


FUNCTIONS
    to_clp(a, frmt='{:1.2f}', arraytype='bmatrix', imstring='j')
        Return a LaTeX array the the clipboard given a numpy array.
        
        Parameters
        ----------
        a         : float array
        frmt      : string
            python 3 formatter, optional-
            https://mkaz.tech/python-string-format.html
        arraytype : string
            latex array type- `bmatrix` default, optional
        imstring : string (optional)
            Character for square root of -1. Usually i or j
        
        Returns
        -------
        out: str
            LaTeX array
        
        See Also
        --------
        array_to_latex
        
        Examples
        --------
        >>> import numpy a

In [5]:
!pip install array-to-latex

  Created wheel for clipboard: filename=clipboard-0.0.4-py3-none-any.whl size=1848 sha256=ba75708c61eeb6841f4cd2b4da447058e7691243f685b51ef9bcfc11e658582c
  Stored in directory: c:\users\sitag\appdata\local\pip\cache\wheels\de\bc\35\fe31f0f559f65513fb90922017715133ed34d148f8eff157f4
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11107 sha256=25c3eaee200c17d3e320896095aa287d0521ebe9c3e9431093d89580e1ba94c6
  Stored in directory: c:\users\sitag\appdata\local\pip\cache\wheels\7f\1a\65\84ff8c386bec21fca6d220ea1f5498a0367883a78dd5ba6122
Successfully built clipboard pyperclip


In [11]:
import array_to_latex
array_to_latex.to_ltx(UA, print_out=True)

\begin{bmatrix}
  0.60 + -0.00j & -0.00 + -0.00j &  0.00 + 0.00j &  0.00 + -0.00j &  0.08 + -0.04j & -0.02 + 0.13j &  0.05 + -0.03j & -0.03 + -0.22j & -0.00 + -0.00j &  0.00 + 0.00j & -0.00 + 0.00j & -0.00 + 0.00j & -0.00 + -0.00j &  0.00 + 0.00j &  0.00 + -0.00j & -0.00 + 0.00j & -0.00 + -0.49j &  0.00 + 0.00j &  0.00 + -0.00j &  0.00 + 0.00j &  0.04 + 0.10j & -0.16 + -0.03j &  0.04 + 0.06j &  0.27 + -0.03j &  0.02 + 0.07j &  0.14 + 0.23j & -0.09 + -0.06j &  0.08 + -0.32j &  0.00 + 0.00j & -0.00 + 0.00j & -0.00 + -0.00j & -0.00 + 0.00j\\
 -0.00 + -0.00j &  0.60 + -0.00j & -0.00 + 0.00j &  0.00 + 0.00j &  0.06 + -0.12j & -0.06 + -0.05j &  0.08 + -0.18j & -0.05 + 0.04j & -0.00 + 0.00j & -0.00 + -0.00j & -0.00 + 0.00j & -0.00 + 0.00j & -0.00 + -0.00j &  0.00 + -0.00j &  0.00 + -0.00j &  0.00 + -0.00j & -0.00 + 0.00j & -0.00 + -0.49j & -0.00 + -0.00j & -0.00 + -0.00j &  0.15 + 0.08j &  0.06 + -0.07j &  0.22 + 0.09j & -0.04 + -0.06j & -0.05 + 0.44j & -0.04 + -0.05j & -0.02 + -0.11j &  0.03

$\pi$

In [12]:
UA

array([[ 6.00000000e-01-5.26274162e-16j, -1.95488394e-16-2.12399699e-16j,
         1.73449611e-16+8.11990190e-17j, ...,
        -4.38690849e-17+2.48470688e-16j, -7.71683383e-17-1.29452765e-16j,
        -3.23109621e-17+2.09977831e-16j],
       [-3.36331432e-17-4.53613828e-16j,  6.00000000e-01-9.00565909e-16j,
        -4.09150390e-17+4.06445194e-17j, ...,
        -3.53505151e-17+8.22832396e-17j, -1.82374770e-16+2.17874149e-16j,
        -1.67272778e-17-5.89160200e-17j],
       [-1.64024263e-16+1.59258490e-16j,  1.89664451e-17+2.79098774e-17j,
         6.00000000e-01-3.61980086e-17j, ...,
         1.46873969e-16+3.15224478e-16j,  1.06665847e-16-1.53408374e-16j,
         7.96595788e-17-2.65168817e-17j],
       ...,
       [-5.28258521e-17-1.97009199e-16j, -9.81917434e-17-5.69331057e-17j,
         1.78428935e-16-2.63860812e-16j, ...,
         6.00000000e-01-1.04069029e-15j,  9.19091176e-17-1.19177652e-16j,
        -3.19727548e-16-2.31110942e-16j],
       [ 5.86038222e-18+9.78927840e-17j, -1.